In [ ]:
import numpy as np
import jax.numpy as jnp
import jax
from jax.typing import ArrayLike
import matplotlib.pyplot as plt

import toflux.src.utils as _utils
import toflux.src.mesher as _mesher
import toflux.src.material as _mat
import toflux.src.bc as _bc

import toflux.src.fe_thermal as _fea_thermal
import toflux.src.fe_fluid as _fea_fluid
import toflux.src.solver as _solv
import toflux.src.viz as _viz


_Ext = _utils.Extent
_FluidField = _fea_fluid.FluidField
_TempField = _fea_thermal.ThermalField

jax.config.update("jax_enable_x64", True)

## Define Geometry and Mesh

In [ ]:
bbox = _mesher.BoundingBox(
  x=_Ext(min=0.0, max=0.2),
  y=_Ext(min=0.0, max=0.02),
)

thermal_mesh = _mesher.GridMesh(
  bounding_box=bbox,
  nel=(100, 20),
  dofs_per_node=1,
  gauss_order=2,
)

fluid_mesh = _mesher.GridMesh(
  bounding_box=bbox,
  nel=(100, 20),
  dofs_per_node=3,
  gauss_order=2,
)

## Material

In [ ]:
fluid_mat = _mat.FluidMaterial(
  mass_density=1.2,
  dynamic_viscosity=1.8e-5,
)

thermal_mat = _mat.ThermalMaterial(
  thermal_conductivity=0.025,
  specific_heat=1006.0,
  mass_density=1.2,
)

## Fluid Boundray Conditions

In [ ]:
reynolds_num = 100.0
char_velocity = reynolds_num * fluid_mat.kinematic_viscosity / (bbox.ly)


def inlet_cond(node_coords):
  left = jnp.all(node_coords[:, 0] <= fluid_mesh.elem_size[0] / 2.0)
  return left


def outlet_cond(node_coords):
  right = jnp.all(node_coords[:, 0] >= bbox.lx - fluid_mesh.elem_size[0] / 2.0)
  return right


def btm_face_cond(node_coords):
  btm = jnp.all(node_coords[:, 1] <= fluid_mesh.elem_size[1] / 2.0)
  return btm


def top_face_cond(node_coords):
  top = jnp.all(node_coords[:, 1] >= bbox.ly - fluid_mesh.elem_size[1] / 2.0)
  return top


inlet_faces = _bc.identify_faces(fluid_mesh, inlet_cond)
n = len(inlet_faces)
u_vel = (_FluidField.U_VEL, char_velocity * jnp.ones(n))
v_vel = (_FluidField.V_VEL, jnp.zeros(n))
inlet_face_val = [u_vel, v_vel]

# outlet condition
outlet_faces = _bc.identify_faces(fluid_mesh, outlet_cond)
n = len(outlet_faces)

v_vel = (_FluidField.V_VEL, jnp.zeros(n))

pres = (_FluidField.PRESSURE, jnp.zeros(n))
outlet_face_val = [v_vel, pres]

# top condition
top_faces = _bc.identify_faces(fluid_mesh, top_face_cond)
n = len(top_faces)

u_vel = (_FluidField.U_VEL, jnp.zeros(n))

v_vel = (_FluidField.V_VEL, jnp.zeros(n))

top_face_val = [u_vel, v_vel]

# bottom condition
bottom_faces = _bc.identify_faces(fluid_mesh, btm_face_cond)
n = len(bottom_faces)

u_vel = (_FluidField.U_VEL, jnp.zeros(n))

v_vel = (_FluidField.V_VEL, jnp.zeros(n))

bottom_face_val = [u_vel, v_vel]

inlet_bc = _bc.DirichletBC(elem_faces=inlet_faces, values=inlet_face_val, name="in")
outlet_bc = _bc.DirichletBC(elem_faces=outlet_faces, values=outlet_face_val, name="out")
top_bc = _bc.DirichletBC(elem_faces=top_faces, values=top_face_val, name="top")
bottom_bc = _bc.DirichletBC(elem_faces=bottom_faces, values=bottom_face_val, name="btm")

fluid_bc_list = [inlet_bc, outlet_bc, top_bc, bottom_bc]

fluid_bc = _bc.process_boundary_conditions(fluid_bc_list, fluid_mesh)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
_viz.plot_grid_mesh(fluid_mesh, ax=ax, colorbar=False)
_viz.plot_bc(fluid_bc_list, fluid_mesh, ax=ax)
plt.show()

## Thermal boundary condition

In [ ]:
def inlet_cond(node_coords):
  left = jnp.all(node_coords[:, 0] <= thermal_mesh.elem_size[0] / 2.0)
  return left


def btm_face_cond(node_coords):
  return jnp.all(node_coords[:, 1] <= 1.05 * thermal_mesh.bounding_box.y.min)


def top_face_cond(node_coords):
  return jnp.all(
    node_coords[:, 1]
    >= thermal_mesh.bounding_box.y.max - thermal_mesh.elem_size[1] / 2.0
  )


inlet_faces = _bc.identify_faces(thermal_mesh, inlet_cond)
n = len(inlet_faces)
tv = (_TempField.TEMPERATURE, 20.0 * jnp.ones(n))
inlet_face_val = [tv]

btm_faces = _bc.identify_faces(thermal_mesh, btm_face_cond)
n = len(btm_faces)
tv = (_TempField.TEMPERATURE, 100.0 * jnp.ones(n))
btm_face_val = [tv]

top_faces = _bc.identify_faces(thermal_mesh, top_face_cond)
n = len(top_faces)
tv = (_TempField.TEMPERATURE, 100.0 * jnp.ones(n))
top_face_val = [tv]


btm_bc = _bc.DirichletBC(elem_faces=btm_faces, values=btm_face_val, name="btm")
inlet_bc = _bc.DirichletBC(elem_faces=inlet_faces, values=inlet_face_val, name="in")
top_bc = _bc.DirichletBC(elem_faces=top_faces, values=top_face_val, name="top")

thermal_bc_list = [btm_bc, inlet_bc, top_bc]
thermal_bc = _bc.process_boundary_conditions(thermal_bc_list, thermal_mesh)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
_viz.plot_grid_mesh(thermal_mesh, ax=ax, colorbar=False)
_viz.plot_bc(thermal_bc_list, thermal_mesh, ax=ax)
plt.show()

# Solver

In [ ]:
solver_settings = {
  "linear": {
    "solver": _solv.LinearSolvers.SCIPY_SPARSE,
    "rtol": 1.0e-3,
    "petsc_solver": {},
  },
  "nonlinear": {"max_iter": 10, "threshold": 1.0e-4},
}

flow_solver = _fea_fluid.FluidSolver(
  mesh=fluid_mesh, bc=fluid_bc, material=fluid_mat, solver_settings=solver_settings
)


thermal_solver = _fea_thermal.FEA(
  mesh=thermal_mesh,
  material=thermal_mat,
  bc=thermal_bc,
  solver_settings=solver_settings,
)

In [ ]:
press_vel = jnp.zeros((fluid_mesh.num_dofs,))
press_vel = press_vel.at[fluid_bc["fixed_dofs"]].set(fluid_bc["dirichlet_values"])


In [ ]:
density = np.zeros((fluid_mesh.num_elems,))
inv_permeability_ext = _utils.Extent(min=0.0, max=122.5)
brinkman_penalty = _mat.compute_ramp_interpolation(
  prop=density,
  ramp_penalty=8.0,
  prop_ext=inv_permeability_ext,
)

In [ ]:
press_vel = _solv.modified_newton_raphson_solve(
  flow_solver, press_vel, brinkman_penalty
)

In [ ]:
u_velocity = press_vel[1 : fluid_mesh.num_dofs : 3]
v_velocity = press_vel[2 : fluid_mesh.num_dofs : 3]

pressure = press_vel[0 : fluid_mesh.num_dofs : 3]
plt.figure()
a = plt.imshow(
  u_velocity.reshape((fluid_mesh.nelx + 1, fluid_mesh.nely + 1)).T,
  cmap="rainbow",
  origin="lower",
)
plt.colorbar(a)
plt.title("U velocity field")


In [ ]:
elem_press_vel = press_vel[fluid_mesh.elem_dof_mat]  # (elems, num_dofs_per_elem)
elem_u_vel = elem_press_vel[:, 1 :: fluid_mesh.nodes.dof_per_node]
elem_v_vel = elem_press_vel[:, 2 :: fluid_mesh.nodes.dof_per_node]

num_vel_dofs_per_elem = fluid_mesh.num_dim * fluid_mesh.elem_template.num_nodes
elem_vel = jnp.zeros((fluid_mesh.num_elems, num_vel_dofs_per_elem))
elem_vel = elem_vel.at[:, 0 :: fluid_mesh.num_dim].set(elem_u_vel)
elem_vel = elem_vel.at[:, 1 :: fluid_mesh.num_dim].set(elem_v_vel)

In [ ]:
plt.figure()
a = plt.imshow(
  elem_u_vel.mean(axis=1).reshape((fluid_mesh.nelx, fluid_mesh.nely)).T,
  cmap="rainbow",
  origin="lower",
)
plt.colorbar(a)

In [ ]:
temp = jnp.zeros((thermal_mesh.num_dofs,))
temp = temp.at[thermal_bc["fixed_dofs"]].set(thermal_bc["dirichlet_values"])


diffusivity_ext = _utils.Extent(
  min=thermal_mat.diffusivity, max=thermal_mat.diffusivity
)

eff_diffusivity = _mat.compute_ramp_interpolation(
  prop=density,
  ramp_penalty=0.01,
  prop_ext=diffusivity_ext,
  mode="concave",
)

temp = _solv.modified_newton_raphson_solve(
  thermal_solver, temp, elem_vel, eff_diffusivity
)

In [ ]:
plt.figure()
a = plt.imshow(
  temp.reshape((fluid_mesh.nelx + 1, fluid_mesh.nely + 1)).T,
  cmap="rainbow",
  origin="lower",
)
plt.colorbar(a)